In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D

# Neural Networks for Images

In [7]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [8]:
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))